In [1]:
from scipy.misc import *
import numpy as np
import pylab as plb
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.integrate import quad
from scipy.integrate import nquad
from scipy.misc import derivative
from ipywidgets import interact, interactive, fixed
from ipywidgets import widgets
import pandas as pd
import emcee
import corner
from astropy import table as Table # For fast and easy reading / writing with tables using numpy library
from galpy.potential import MiyamotoNagaiPotential, NFWPotential, RazorThinExponentialDiskPotential, BurkertPotential # GALPY potentials

In [2]:
# In[ ]:

tt=Table.Table.read('coordinates.txt', format='ascii.tab') # importando los datos de las imágenes

#Importando coordenadas de las imágenes deflectadas
theta1=tt['theta1'] 
theta2=tt['theta2']

theta=np.zeros(len(theta1),float)
for i in range(len(theta1)):
    theta[i]=np.sqrt(theta1[i]**2+theta2[i]**2)

tt=Table.Table.read('alpha.txt', format='ascii.tab') # importando los datos de las imágenes los ángulos en la circunferencia para la fuente circular
#Imprtando los valores de alpha
alpha=tt['alpha']

FC = np.pi/(180*3600) #conversion factor between arcs and radians
R = 0.03
r = R*np.pi/(180*3600)
H = -0.0356
h = H*np.pi/(180*3600)
K = 0.00593
K = K*np.pi/(180*3600)
    
Beta1 = r*np.cos(alpha)+h
Beta2 = r*np.sin(alpha)+K


#Definiendo la lente y sus parámetros principales
D_ds = 1179.6e3
D_d = 497.6e3
D_s = 1510.2e3
SIGMA_CRIT = 4285.3e6 #Sigma crítico para la convergencia

In [3]:
def THETA(R_S,m_0, SIGMA_0, H_R, B,MASS):
    GRADPOT1disk_exp = np.zeros((len(theta1)), float)
    GRADPOT2disk_exp = np.zeros((len(theta1)), float)
    GRADPOT1 = np.zeros((len(theta1)), float)
    GRADPOT2 = np.zeros((len(theta1)), float)
    THETA1 = np.zeros((len(theta1)), float)
    THETA2 = np.zeros((len(theta1)), float)
    GRADPOT1nfw = np.zeros((len(theta1)), float)
    GRADPOT2nfw = np.zeros((len(theta1)), float)
    GRADPOT1MN = np.zeros((len(theta1)), float)
    GRADPOT2MN = np.zeros((len(theta1)), float)
    THETA_teor = np.zeros((len(theta1)), float)
    def POTDEFnfw1(x):
        def integ(TheTa, theta):
            R = D_d*TheTa
            NFW_p = NFWPotential(amp=m_0, a=R_S, normalize=False)
            Sigma = NFW_p.dens(R,0.)
            kappa = Sigma/SIGMA_CRIT
            return 2*TheTa*np.log(THETA/TheTa)*kappa/(SIGMA_CRIT**2)
        THETA = np.sqrt(x**2 + theta2[l]**2)
        x = quad(integ, 0, theta[l], limit=100, args=(theta[l]))[0]
        return x
    def POTDEFnfw2(x):
        def integ(TheTa, theta):
            R = D_d*TheTa
            NFW_p = NFWPotential(amp=m_0, a=R_S, normalize=False)
            Sigma = NFW_p.dens(R,0.)
            kappa = Sigma/SIGMA_CRIT
            return 2*TheTa*np.log(THETA/TheTa)*kappa/(SIGMA_CRIT**2)
        THETA = np.sqrt(x**2 + theta1[l]**2)
        x = quad(integ, 0, theta[l], limit=100, args=(theta[l]))[0]
        return x
    def MN1(x):
        def integ(TheTa, theta):
            R = D_d*TheTa
            MN_Bulge_p= MiyamotoNagaiPotential(amp=MASS,a=0,b=B,normalize=False)
            Sigma = MN_Bulge_p.dens(R,0.)
            kappa = Sigma/SIGMA_CRIT
            return 2*TheTa*np.log(THETA/TheTa)*kappa/(SIGMA_CRIT**2)
        THETA = np.sqrt(x**2 + theta2[l]**2)
        x = quad(integ, 0, theta[l], limit=100, args=(theta[l]))[0]
        return x
    def MN2(x):
        def integ(TheTa, theta):
            R = D_d*TheTa
            MN_Bulge_p= MiyamotoNagaiPotential(amp=MASS,a=0,b=B,normalize=False)
            Sigma = MN_Bulge_p.dens(R,0.)
            kappa = Sigma/SIGMA_CRIT
            return 2*TheTa*np.log(THETA/TheTa)*kappa/(SIGMA_CRIT**2)
        THETA = np.sqrt(x**2 + theta1[l]**2)
        x = quad(integ, 0, theta[l], limit=100, args=(theta[l]))[0]
        return x
    def POTDEFdisk_exp1(x):
        def integ(TheTa, theta):
            Sigma = SIGMA_0*np.exp(-D_d*TheTa/H_R) #Volumetric density
            return 2*TheTa*np.log(THETA/TheTa)*Sigma/(SIGMA_CRIT**3)
        THETA = np.sqrt(x**2 + theta2[l]**2)
        x = quad(integ, 0, theta[l], limit=100, args=(theta))[0]
        return x
    def POTDEFdisk_exp2(x):
        def integ(TheTa, theta):
            Sigma = SIGMA_0*np.exp(-D_d*TheTa/H_R) #Volumetric density
            return 2*TheTa*np.log(THETA/TheTa)*Sigma/(SIGMA_CRIT**3)
        THETA = np.sqrt(x**2 + theta1[l]**2)
        x = quad(integ, 0, theta[l], limit=100, args=(theta))[0]
        return x
    for l in range(len(theta1)):
        GRADPOT1nfw[l]= derivative(POTDEFnfw1, theta1[l], dx=1e-9, order=7)
        GRADPOT2nfw[l]= derivative(POTDEFnfw2, theta2[l], dx=1e-9, order=7)
    for l in range(len(theta1)):
        GRADPOT1disk_exp[l]= derivative(POTDEFdisk_exp1, theta1[l], dx=1e-9, order=7)
        GRADPOT2disk_exp[l]= derivative(POTDEFdisk_exp2, theta2[l], dx=1e-9, order=7)
    for l in range(len(theta1)):
        GRADPOT1MN[l]= derivative(MN1, theta1[l], dx=1e-9, order=7)
        GRADPOT2MN[l]= derivative(MN2, theta2[l], dx=1e-9, order=7)    
    for l in range(len(theta1)):
        GRADPOT1[l]=(SIGMA_CRIT**2)*(GRADPOT1nfw[l]+GRADPOT1disk_exp[l]+GRADPOT1MN[l])
        GRADPOT2[l]=(SIGMA_CRIT**2)*(GRADPOT2nfw[l]+GRADPOT2disk_exp[l]+GRADPOT2MN[l])
    for l in range(len(theta1)):
        THETA1[l] = Beta1[l]+GRADPOT1[l]
        THETA2[l] = Beta2[l]+GRADPOT2[l]
    #Graphics of source and images
    fig = plt.figure()
#    ax = fig.add_subplot(111)
    plt.rcParams['figure.figsize'] =(10,10)
    plb.plot(Beta1/FC, Beta2/FC, '--r')
    plb.plot(theta1/FC, theta2/FC, 'ob')
    plb.plot(THETA1/FC, THETA2/FC, 'og')
    plb.xlim(-2.5,2.5)
    plb.ylim(-2.5,2.5)
    plb.legend(['Source', 'Observational data', 'model values'], loc='upper right', fontsize=15)
 #   ax.set_aspect('equal', adjustable='box')

In [4]:
interact(THETA,R_S=widgets.FloatSlider(min=0.1, max=60.1, step=1, value=0.1),
        m_0=widgets.FloatSlider(min=0.1e11, max=1e12, step=0.1e11, value=0.6),
        H_R=widgets.FloatSlider(min=2, max=30, step=0.2, value=3),
        SIGMA_0 = widgets.FloatSlider(min=1e8, max=1e10, step=0.5e8, value=1e8),
        B=widgets.FloatSlider(min=0.1, max=2, step=0.02, value=0.3),
        MASS=widgets.FloatSlider(min=0.1e10, max=2e10, step=0.05e10, value=0.5e10));

interactive(children=(FloatSlider(value=0.1, description='R_S', max=60.1, min=0.1, step=1.0), FloatSlider(valu…